# Line chart (benchmark analysis)

In [7]:

import pandas as pd
import vizro.models as vm
import vizro.plotly.express as px
from vizro import Vizro
from vizro.tables import dash_ag_grid
from vizro.actions import export_data, filter_interaction
import plotly.graph_objects as go
from vizro.models.types import capture

Vizro._reset()

gapminder = pd.read_csv('uni.csv',
                        header=0,
                        usecols=["University Name", "Arts and humanities ranking THE2024", "Engineering & Technology ranking THE24", "Natural science rankings QS24", "Life science/ medical rankings QS24", "Economics & Business Rankings THE24"])

@capture("graph")
def barchart(data_frame, x, y, title=None, text=None):
    fig_bar = px.bar(data_frame=data_frame, x=x, y=y, title=title, text=text)
    
    fig_bar.update_layout(barmode='stack', xaxis={'categoryorder':'total ascending'})
    fig_bar.update_layout(
    title=go.layout.Title(
    text="<sup>Smaller number means higher national ranking</sup>",
    xref="paper",
    x=0
        ),)

    return fig_bar


def create_subject():
    """Function returns a page to perform analysis on university level."""

    page_subject = vm.Page(
        title="Uni rankings by subject",
        description="Discovering how different NZ universities are ranked through the years",
        layout=vm.Layout( grid=[[1, 1, 1, 1, 1]] + [[0, 0, 0, 0, 0]] * 4,
            row_min_height="100px",
            row_gap="24px",),
        components=[
            vm.Graph(
                id="barchart",
                figure=barchart(
                    gapminder,
                    x="University Name",
                    y="Arts and humanities ranking THE2024",
                    text="Arts and humanities ranking THE2024"
                ),
            ),
            vm.Card(
            text="""
               scroll down! 
               ### National rankings of universities based on subject field
               
                 - Subject fields to choose from: 
                    - 🏛️ Arts and humanities: Art, History, Philosophy, Literature, Classics, Media Studies, Film studies 
                    - ⚙️ Engineering & Technology: Electronic, Computer engineering, Biomedical engineering, Computer Science, IT, Food Technology
                    - 🔬 Natural science: Physics, Chemistry, Biology 
                    - 🩺 Life science/ medical: Pharmaceuticals, Biotechnology, Biomedical Science 
                    - 🏭 Economics & Business: Economics, Marketing, Finance
                    
                >
                >  Choose subject field in selector on your left. 
                >
             
  
            
            """,
        ),
        ],
        controls=[
            vm.Parameter(
                targets=["barchart.y"],
                selector=vm.Dropdown(
                    options=["Arts and humanities ranking THE2024", "Engineering & Technology ranking THE24", "Natural science rankings QS24", "Life science/ medical rankings QS24", "Economics & Business Rankings THE24"], multi=False, value="Arts and humanities ranking THE2024", title="Choose subject field"
                ),
            ),
             vm.Parameter(
                targets=["barchart.text"],
                selector=vm.Dropdown(
                    options=["Arts and humanities ranking THE2024", "Engineering & Technology ranking THE24", "Natural science rankings QS24", "Life science/ medical rankings QS24", "Economics & Business Rankings THE24"], multi=False, value="Arts and humanities ranking THE2024", title="Choose subject field to see specific rankings on the bars"
                ),
            ),
        ],
    )

    return page_subject


IS_JUPYTERLAB = 'true'



dashboard = vm.Dashboard(
    pages=[
        create_subject(),
        
    ],
    navigation=vm.Navigation(
        nav_selector=vm.NavBar(
            items=[
                vm.NavLink(
                    label="Rankings of subject field",
                    pages=["Uni rankings by subject"],
                    icon="Auto Stories",
                ),
            ]
        ),
    ),
)

if not IS_JUPYTERLAB:
    app = Vizro().build(dashboard)
    server = app.dash.server
    
    if __name__ == "__main__":  
        app.run(port=8080)
else:
    Vizro(assets_folder="assets").build(dashboard).run(port=8082)

